In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import BasicIterativeMethod
from art.estimators.classification import PyTorchClassifier

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
columns = ['Dataset', 'Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1', 'Weighted Precision', 'Weighted Recall', 'Weighted F1', 'Macro AUC', 'Weighted AUC', 'TPR', 'FNR', 'TNR', 'FPR']
results_df = pd.DataFrame(columns=columns)

In [17]:
def calculate_performance_metrics(X_test, y_true, model, results_df, name):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}\nAUC: {macro_auc}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}\nAUC: {weighted_auc}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            'Dataset': name,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Macro Recall': recall_macro,
            'Macro F1': f1_macro,
            'Macro AUC': macro_auc,
            'Weighted Precision': precision_weighted,
            'Weighted Recall': recall_weighted,
            'Weighted F1': f1_weighted,
            'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }

        new_row_df = pd.DataFrame([new_row])  # Convert your new row into a DataFrame
        results_df = pd.concat([results_df, new_row_df], ignore_index=True)
        return results_df

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_train = np.load("/home/jovyan/UNSW/dataset/preprocessed/x_train.npy")
y_train = np.load("/home/jovyan/UNSW/dataset/preprocessed/y_train.npy")
x_val = np.load("/home/jovyan/UNSW/dataset/preprocessed/x_val.npy")
y_val = np.load("/home/jovyan/UNSW/dataset/preprocessed/y_val.npy")
x_test = np.load("/home/jovyan/UNSW/dataset/preprocessed/x_test.npy")
y_test = np.load("/home/jovyan/UNSW/dataset/preprocessed/y_test.npy")

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/UNSW/model/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [18]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.01, max_iter = 10)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [19]:
np.save("/home/jovyan/UNSW/dataset/with_attack/pytorch/BIM_001.npy", x_test_adv)

In [20]:
results_df = calculate_performance_metrics(x_test_adv, y_test, model, results_df, "001")

Accuracy: 0.8905732258855601

macro
Precision: 0.8697604936669707
Recall: 0.9105092863397444
F1 Score: 0.88171001444566
AUC: 0.9799952643954238

weighted
Precision: 0.9101359575565199
Recall: 0.8905732258855601
F1 Score: 0.8934069199339241
AUC: 0.9799952643954238

Mean FNR: 0.08949071366025567
Mean TNR: 0.9105092863397444
Mean FPR: 0.08949071366025567
Mean TPR: 0.9105092863397444


/tmp/ipykernel_18632/3177275996.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row_df], ignore_index=True)


In [21]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Macro AUC,Weighted AUC,TPR,FNR,TNR,FPR
0,001,0.890573,0.86976,0.910509,0.88171,0.910136,0.890573,0.893407,0.979995,0.979995,0.910509,0.089491,0.910509,0.089491


In [22]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.05, max_iter = 10)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [23]:
np.save("/home/jovyan/UNSW/dataset/with_attack/pytorch/BIM_005.npy", x_test_adv)

In [24]:
results_df = calculate_performance_metrics(x_test_adv, y_test, model, results_df, "005")

Accuracy: 0.852818222777331

macro
Precision: 0.8353743307898338
Recall: 0.8795748829584372
F1 Score: 0.8435273627652483
AUC: 0.974101680418776

weighted
Precision: 0.8853455470342398
Recall: 0.852818222777331
F1 Score: 0.8573009985847806
AUC: 0.974101680418776

Mean FNR: 0.12042511704156277
Mean TNR: 0.8795748829584372
Mean FPR: 0.12042511704156277
Mean TPR: 0.8795748829584372


In [25]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Macro AUC,Weighted AUC,TPR,FNR,TNR,FPR
0,001,0.890573,0.869760,0.910509,0.881710,0.910136,0.890573,0.893407,0.979995,0.979995,0.910509,0.089491,0.910509,0.089491
1,005,0.852818,0.835374,0.879575,0.843527,0.885346,0.852818,0.857301,0.974102,0.974102,0.879575,0.120425,0.879575,0.120425


In [26]:
attack = BasicIterativeMethod(estimator=classifier, eps=0.1, max_iter = 10)
X_test_adv = attack.generate(x=x_test)

In [27]:
np.save("/home/jovyan/UNSW/dataset/with_attack/pytorch/BIM_01.npy", x_test_adv)

In [28]:
results_df = calculate_performance_metrics(x_test_adv, y_test, model, results_df, "01")

Accuracy: 0.852818222777331

macro
Precision: 0.8353743307898338
Recall: 0.8795748829584372
F1 Score: 0.8435273627652483
AUC: 0.974101680418776

weighted
Precision: 0.8853455470342398
Recall: 0.852818222777331
F1 Score: 0.8573009985847806
AUC: 0.974101680418776

Mean FNR: 0.12042511704156277
Mean TNR: 0.8795748829584372
Mean FPR: 0.12042511704156277
Mean TPR: 0.8795748829584372


In [29]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Macro AUC,Weighted AUC,TPR,FNR,TNR,FPR
0,001,0.890573,0.869760,0.910509,0.881710,0.910136,0.890573,0.893407,0.979995,0.979995,0.910509,0.089491,0.910509,0.089491
1,005,0.852818,0.835374,0.879575,0.843527,0.885346,0.852818,0.857301,0.974102,0.974102,0.879575,0.120425,0.879575,0.120425
2,01,0.852818,0.835374,0.879575,0.843527,0.885346,0.852818,0.857301,0.974102,0.974102,0.879575,0.120425,0.879575,0.120425


In [30]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.2, max_iter = 10)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [31]:
np.save("/home/jovyan/UNSW/dataset/with_attack/pytorch/BIM_02.npy", x_test_adv)

In [32]:
results_df = calculate_performance_metrics(x_test_adv, y_test, model, results_df, "02")

Accuracy: 0.6006809588173901

macro
Precision: 0.6514689060542215
Recall: 0.6645890829639437
F1 Score: 0.5990811968740688
AUC: 0.8435152408404728

weighted
Precision: 0.7295712966421972
Recall: 0.6006809588173901
F1 Score: 0.6082298535418116
AUC: 0.8435152408404728

Mean FNR: 0.33541091703605636
Mean TNR: 0.6645890829639437
Mean FPR: 0.33541091703605636
Mean TPR: 0.6645890829639437


In [33]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.3, max_iter = 10)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [34]:
np.save("/home/jovyan/UNSW/dataset/with_attack/pytorch/BIM_03.npy", x_test_adv)

In [35]:
results_df = calculate_performance_metrics(x_test_adv, y_test, model, results_df, "03")

Accuracy: 0.4321636126176992

macro
Precision: 0.5051935645136825
Recall: 0.5050186376942662
F1 Score: 0.43195327969576636
AUC: 0.6891533800352412

weighted
Precision: 0.5711923686490458
Recall: 0.4321636126176992
F1 Score: 0.4280046437715958
AUC: 0.6891533800352412

Mean FNR: 0.49498136230573375
Mean TNR: 0.5050186376942662
Mean FPR: 0.49498136230573375
Mean TPR: 0.5050186376942662


In [36]:
results_df

,Dataset,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Macro AUC,Weighted AUC,TPR,FNR,TNR,FPR
0,001,0.890573,0.869760,0.910509,0.881710,0.910136,0.890573,0.893407,0.979995,0.979995,0.910509,0.089491,0.910509,0.089491
1,005,0.852818,0.835374,0.879575,0.843527,0.885346,0.852818,0.857301,0.974102,0.974102,0.879575,0.120425,0.879575,0.120425
2,01,0.852818,0.835374,0.879575,0.843527,0.885346,0.852818,0.857301,0.974102,0.974102,0.879575,0.120425,0.879575,0.120425
3,02,0.600681,0.651469,0.664589,0.599081,0.729571,0.600681,0.608230,0.843515,0.843515,0.664589,0.335411,0.664589,0.335411
4,03,0.432164,0.505194,0.505019,0.431953,0.571192,0.432164,0.428005,0.689153,0.689153,0.505019,0.494981,0.505019,0.494981


In [37]:
results_df.to_csv("/home/jovyan/UNSW/BIM.csv", index=False)